In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
from fake_useragent import UserAgent

In [3]:
ua = UserAgent()
headers = {
    "User-Agent": ua.random,
    "Accept-Language": "en-US,en;q=0.5",
}

In [4]:
url = "https://www.amazon.com/s?k=laptop&crid=2RMA65OL0L9CB&sprefix=laptop%2Caps%2C452&ref=nb_sb_noss_1"

In [ ]:
response = requests.get(url, headers=headers)

In [6]:
soup = BeautifulSoup(response.text, "html.parser")

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js a-touch a-mobile" data-19ax5a9jf="mongoose" lang="en-us">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta content="width=device-width, maximum-scale=2, minimum-scale=1, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta charset="utf-8"/>
  <!-- sp:end-feature:head-start -->
  <!-- sp:feature:csm:head-open-part1 -->
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <!-- sp:end-feature:csm:head-open-part1 -->
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:end-feature:cs-optimization -->
  <!-- sp:feature:csm:head-open-part2 -->
  <script type="text/javascript">
   window.ue_ihb = (wind

In [17]:
products = soup.find_all("div", {"data-component-type": "s-search-result"})

for product in products:
    title = product.find("h2", class_="a-text-normal")
    price = product.find("span", class_="a-offscreen")
    rating = product.find("span", class_="a-icon-alt")
    image = product.find("img") 

    print("Title:", title.text.strip() if title else "N/A")
    print("Price:", price.text.strip() if price else "N/A")
    print("Rating:", rating.text.strip() if rating else "N/A")
    print("Image URL:", image["src"] if image else "N/A")

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to scrape a page
def scrape_page(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    products = soup.find_all("div", {"data-component-type": "s-search-result"})
    
    data = []
    for product in products:
        title = product.find("h2", class_="a-text-normal")  # Changed to span instead of h2
        price = product.find("span", class_="a-offscreen")
        rating = product.find("span", class_="a-icon-alt")
        image = product.find("img")  # Fixed variable name

        data.append([
            title.text.strip() if title else "N/A",
            price.text.strip() if price else "N/A",
            rating.text.strip() if rating else "N/A",
            image["src"] if image else "N/A"  # Fixed incorrect quotation marks
        ])

    return data, soup

In [7]:
# Base URL (First Page)
base_url = "https://www.amazon.com/s?k=laptop&crid=2RMA65OL0L9CB&sprefix=laptop%2Caps%2C452&ref=nb_sb_noss_1"

# Store all product data
all_data = []

# Maximum pages to scrape
max_pages = 20
current_page = 1  
url = base_url  

while current_page <= max_pages:
    print(f"Scraping page {current_page}...")
    page_data, soup = scrape_page(url)
    all_data.extend(page_data)

    # Find "Next Page" button
    next_page = soup.find("a", class_="s-pagination-next")
    if next_page and "href" in next_page.attrs:
        next_url = "https://www.amazon.com/s?k=laptop&crid=2RMA65OL0L9CB&sprefix=laptop%2Caps%2C452&ref=nb_sb_noss_1" + next_page["href"]
        url = next_url
        current_page += 1
        time.sleep(5)
    else:
        break

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...


In [ ]:
# Convert to DataFrame
df = pd.DataFrame(all_data, columns=["Title", "Price", "Rating", "Image URL"])
df

In [9]:
df.to_csv("amazon_laptops.csv", index=False)
print("Scraping complete. Data saved as amazon_laptops.csv.")

Scraping complete. Data saved as amazon_laptops.csv.


In [ ]:
df.to_csv(r"C:\Users\holar\Documents\Excel Project\Excel Projects\amazon_laptops.csv", index=False)

In [1]:
import pandas as pd

In [2]:
# Load dataset
df = pd.read_csv(r"C:\Users\holar\Documents\Excel Project\Excel Projects\amazon_laptops.csv")

In [3]:
# Define categories and keywords
categories = {
    "Gaming": ["gaming", "RTX", "GeForce", "high-performance", "RGB"],
    "Office": ["business", "office", "lightweight", "professional"],
    "Student": ["student", "budget", "Chromebook", "affordable"]
}

In [4]:
# Function to assign categories
def categorize_laptop(title):
    title_lower = title.lower()
    for category, keywords in categories.items():
        if any(keyword in title_lower for keyword in keywords):
            return category
    return "Other"  # If no keywords match, label as "Other"

In [5]:
# Apply function to dataset
df["Category"] = df["Title"].apply(categorize_laptop)

In [6]:
# Count the number of products in each category
category_counts = df["Category"].value_counts()
print(category_counts)

Category
Other      209
Office     139
Gaming      71
Student      7
Name: count, dtype: int64


In [7]:
# Save updated dataset
df.to_csv("categorized_laptops.csv", index=False)

In [8]:
df.to_csv(r"C:\Users\holar\Documents\Excel Project\Excel Projects\categorized_laptops.csv", index=False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\holar\Documents\Excel Project\Excel Projects\amazon_laptops1B.csv")

In [3]:
df

,Title,Brand,RAM,Price,Ratings,Image URL,Category
0,Gaming Laptop Laptop Computer With Amd Ryzen 7...,Gaming,16GB,$495,5,https://m.media-amazon.com/images/I/81jFW4cV7b...,Gaming
1,Hp 14 Laptop Intel Celeron N4020 4 Gb Ram 64 G...,HP,Others,$178,4,https://m.media-amazon.com/images/I/815uX7wkOZ...,Other
2,Acer Aspire 3 A31524Pr7Vh Slim Laptop 156 Full...,Acer,8GB,$300,4,https://m.media-amazon.com/images/I/61gKkYQn6l...,Other
3,Hp Portable Laptop Include 1 Year Microsoft 36...,HP,16GB,$239,4,https://m.media-amazon.com/images/I/61DVg3ECW-...,Office
4,Lenovo Ideapad Slim 3 Chromebook 2024 Lightwei...,Lenovo,4GB,$181,4,https://m.media-amazon.com/images/I/61BRKJYlOS...,Office
...,...,...,...,...,...,...,...
313,Gaming Laptop With Amd Ryzen 7 5825U Processor...,Gaming,16GB,"$1,700",4,https://m.media-amazon.com/images/I/71zYBSryFo...,Gaming
314,Samsung Chromebook Plus V2 2In1 Laptop 4Gb Ram...,Samsung,4GB,$245,4,https://m.media-amazon.com/images/I/81mX-4s+gu...,Other
315,Asus 2023 Vivobook 17X Laptop 173 Fhd Display ...,ASUS,16GB,$902,5,https://m.media-amazon.com/images/I/71gLz5gSDj...,Other
316,15 156 Fhd 1920 X 1080 Display Laptop Computer...,Others,8GB,$299,5,https://m.media-amazon.com/images/I/71HHbvhu2W...,Office


In [4]:
# Function to extract brand and model from the Title, and clean unnecessary parts
def clean_title(title):
    title = str(title).strip()  # Ensure it's a string and strip any leading/trailing spaces

In [8]:
 # Define a regex pattern to extract brand and model
pattern = r"([A-Za-z]+(?:\s[A-Za-z]+)*)\s([A-Za-z0-9\-]+(?:\s[A-Za-z0-9\-]+)*)"

In [18]:
def clean_title(title):
    match = re.search(pattern, title)
    if match:
        # Extract brand and model if matched
        brand = match.group(1).strip()
        model = match.group(2).strip()
        # Combine brand and model into a clean title
        return f"{brand} {model}"
    else:
        # If no match, return the title as is
        return title.strip()

In [21]:
import re
import pandas as pd

In [50]:
# Apply function to 'title' column
df["Title"] = df["Title"].apply(clean_title)

In [51]:
df.head()

,Title,Brand,RAM,Price,Category,words,brand,model
0,Gaming Laptop Laptop Computer With Amd Ryzen 7...,Gaming,16GB,$495,Gaming,"[Gaming, Laptop, Laptop, Computer, With, Amd, ...",Gaming,Laptop Laptop Computer With Amd Ryzen 7 5700U ...
1,Hp 14 Laptop Intel Celeron N4020 4 Gb Ram 64 G...,Hp,Others,$178,Other,"[Hp, 14, Laptop, Intel, Celeron, N4020, 4, Gb,...",Hp,14 Laptop Intel Celeron N4020 4 Gb Ram 64 Gb S...
2,Acer Aspire 3 A31524Pr7Vh Slim Laptop 156 Full...,Acer,8GB,$300,Other,"[Acer, Aspire, 3, A31524Pr7Vh, Slim, Laptop, 1...",Acer,Aspire 3 A31524Pr7Vh Slim Laptop 156 Full Hd I...
3,Hp Portable Laptop Include 1 Year Microsoft 36...,Hp,16GB,$239,Office,"[Hp, Portable, Laptop, Include, 1, Year, Micro...",Hp,Portable Laptop Include 1 Year Microsoft 365 1...
4,Lenovo Ideapad Slim 3 Chromebook 2024 Lightwei...,Lenovo,4GB,$181,Office,"[Lenovo, Ideapad, Slim, 3, Chromebook, 2024, L...",Lenovo,Ideapad Slim 3 Chromebook 2024 Lightweight Lap...


In [52]:
# Remove unnecessary words
words_to_remove = [
    "computer", "notebook", "portable", "gaming", "full hd", 
        "lightweight", "include", "year", "microsoft", "windows", "chromebook", "touchscreen"
    ]

In [33]:
 # Remove extra words using regex
pattern = r"\b(" + "|".join(words_to_remove) + r")\b"
df["Title"] = df["Title"].str.replace(pattern, "", regex=True).str.strip()

In [38]:
# Extract brand
df["Brand"] = df["Title"].str.split().str[0].fillna("Unknown")

In [41]:
 # Extract model (remaining words)
df["words"] = df["Title"].apply(lambda x: x.split() if isinstance(x, str) else [])

In [42]:
df["brand"] = df["words"].apply(lambda x: x[0] if len(x) > 0 else "Unknown")
df["model"] = df["words"].apply(lambda x: " ".join(x[1:]) if len(x) > 1 else "Unknown")

In [43]:
# Apply cleaning function to 'Title' column
df["Title"] = df["Title"].apply(clean_title)

In [49]:
df.head()

,Title,Brand,RAM,Price,Category,words,brand,model
0,Gaming Laptop Laptop Computer With Amd Ryzen 7...,Gaming,16GB,$495,Gaming,"[Gaming, Laptop, Laptop, Computer, With, Amd, ...",Gaming,Laptop Laptop Computer With Amd Ryzen 7 5700U ...
1,Hp 14 Laptop Intel Celeron N4020 4 Gb Ram 64 G...,Hp,Others,$178,Other,"[Hp, 14, Laptop, Intel, Celeron, N4020, 4, Gb,...",Hp,14 Laptop Intel Celeron N4020 4 Gb Ram 64 Gb S...
2,Acer Aspire 3 A31524Pr7Vh Slim Laptop 156 Full...,Acer,8GB,$300,Other,"[Acer, Aspire, 3, A31524Pr7Vh, Slim, Laptop, 1...",Acer,Aspire 3 A31524Pr7Vh Slim Laptop 156 Full Hd I...
3,Hp Portable Laptop Include 1 Year Microsoft 36...,Hp,16GB,$239,Office,"[Hp, Portable, Laptop, Include, 1, Year, Micro...",Hp,Portable Laptop Include 1 Year Microsoft 365 1...
4,Lenovo Ideapad Slim 3 Chromebook 2024 Lightwei...,Lenovo,4GB,$181,Office,"[Lenovo, Ideapad, Slim, 3, Chromebook, 2024, L...",Lenovo,Ideapad Slim 3 Chromebook 2024 Lightweight Lap...


In [45]:
df["Title"]

0      Gaming Laptop Laptop Computer With Amd Ryzen 7...
1      Hp 14 Laptop Intel Celeron N4020 4 Gb Ram 64 G...
2      Acer Aspire 3 A31524Pr7Vh Slim Laptop 156 Full...
3      Hp Portable Laptop Include 1 Year Microsoft 36...
4      Lenovo Ideapad Slim 3 Chromebook 2024 Lightwei...
                             ...                        
313    Gaming Laptop With Amd Ryzen 7 5825U Processor...
314    Samsung Chromebook Plus V2 2In1 Laptop 4Gb Ram...
315    Asus 2023 Vivobook 17X Laptop 173 Fhd Display ...
316    Fhd 1920 X 1080 Display Laptop Computer For Co...
317    Jumper Laptop 24Gb Lpddr4X Ram 1024Gb Nvme Ssd...
Name: Title, Length: 318, dtype: object

In [53]:
df["Ratings"] = pd.read_csv(r"C:\Users\holar\Documents\Excel Project\Excel Projects\amazon_laptops1B.csv")["Ratings"]

In [54]:
df.head()

,Title,Brand,RAM,Price,Category,words,brand,model,Ratings
0,Gaming Laptop Laptop Computer With Amd Ryzen 7...,Gaming,16GB,$495,Gaming,"[Gaming, Laptop, Laptop, Computer, With, Amd, ...",Gaming,Laptop Laptop Computer With Amd Ryzen 7 5700U ...,5
1,Hp 14 Laptop Intel Celeron N4020 4 Gb Ram 64 G...,Hp,Others,$178,Other,"[Hp, 14, Laptop, Intel, Celeron, N4020, 4, Gb,...",Hp,14 Laptop Intel Celeron N4020 4 Gb Ram 64 Gb S...,4
2,Acer Aspire 3 A31524Pr7Vh Slim Laptop 156 Full...,Acer,8GB,$300,Other,"[Acer, Aspire, 3, A31524Pr7Vh, Slim, Laptop, 1...",Acer,Aspire 3 A31524Pr7Vh Slim Laptop 156 Full Hd I...,4
3,Hp Portable Laptop Include 1 Year Microsoft 36...,Hp,16GB,$239,Office,"[Hp, Portable, Laptop, Include, 1, Year, Micro...",Hp,Portable Laptop Include 1 Year Microsoft 365 1...,4
4,Lenovo Ideapad Slim 3 Chromebook 2024 Lightwei...,Lenovo,4GB,$181,Office,"[Lenovo, Ideapad, Slim, 3, Chromebook, 2024, L...",Lenovo,Ideapad Slim 3 Chromebook 2024 Lightweight Lap...,4


In [56]:
df_temp = df.drop(columns=["words"])

In [57]:
df_temp = df.drop(columns=["brand"])

In [58]:
df_temp = df.drop(columns=["model"])

In [62]:
print(df_temp.head())

                                               Title   Brand     RAM Price  \
0  Gaming Laptop Laptop Computer With Amd Ryzen 7...  Gaming   16GB   $495   
1  Hp 14 Laptop Intel Celeron N4020 4 Gb Ram 64 G...      Hp  Others  $178   
2  Acer Aspire 3 A31524Pr7Vh Slim Laptop 156 Full...    Acer    8GB   $300   
3  Hp Portable Laptop Include 1 Year Microsoft 36...      Hp   16GB   $239   
4  Lenovo Ideapad Slim 3 Chromebook 2024 Lightwei...  Lenovo    4GB   $181   

  Category                                              words   brand  Ratings  
0   Gaming  [Gaming, Laptop, Laptop, Computer, With, Amd, ...  Gaming        5  
1    Other  [Hp, 14, Laptop, Intel, Celeron, N4020, 4, Gb,...      Hp        4  
2    Other  [Acer, Aspire, 3, A31524Pr7Vh, Slim, Laptop, 1...    Acer        4  
3   Office  [Hp, Portable, Laptop, Include, 1, Year, Micro...      Hp        4  
4   Office  [Lenovo, Ideapad, Slim, 3, Chromebook, 2024, L...  Lenovo        4  


In [63]:
df.drop(columns=["words"], inplace=True)

In [64]:
df.drop(columns=["brand"], inplace=True)

In [65]:
df.drop(columns=["model"], inplace=True)

In [67]:
df.drop(columns=["Title"], inplace=True)

In [68]:
df.head()

,Brand,RAM,Price,Category,Ratings
0,Gaming,16GB,$495,Gaming,5
1,Hp,Others,$178,Other,4
2,Acer,8GB,$300,Other,4
3,Hp,16GB,$239,Office,4
4,Lenovo,4GB,$181,Office,4


In [70]:
df["Brand"] = df["Brand"].str.upper()

In [72]:
df.to_csv("cleaned_amazon_laptops.csv", index=False)

In [73]:
df.to_csv(r"C:\Users\holar\Documents\Excel Project\Excel Projects\cleaned_amazon_laptops.csv", index=False)